## Intelligent  Pathfinding System

### import libaries

In [205]:
import heapq   # Priority queue for A* algorithm
import numpy as np   # Numerical arrays and math operations  
import plotly.express as px  # Visualization (plotting grids/paths)  
import time  # Timing algorithm performance 
from collections import deque  # Efficient FIFO queue for BFS  
import random  # Random grid generation 

### Cell Class Definition

In [215]:
class Cell:
    def __init__(self, x, y):
        self.x = x  # Column position (0-indexed)
        self.y = y # Row position (0-indexed)
        self.type = 'empty'  # start, goal, obstacle, empty
        self.parent = None   # Path reconstruction 
        self.g_cost = float('inf')   # Cost from start node

    def __repr__(self):
        return f"Cell({self.x}, {self.y}, {self.type})"

### Grid Creation Function

In [216]:
def create_grid(size=10, obstacle_prob=0.2):
    # Initialize grid with empty cells
    grid = [[Cell(x, y) for y in range(size)] for x in range(size)]
    
    # Set start (top-left) and goal (bottom-right)
    grid[0][0].type = 'start'
    grid[size-1][size-1].type = 'goal'
    
    # Randomly place obstacles (avoiding start/goal)
    for x in range(size):
        for y in range(size):
            if (x, y) not in [(0, 0), (size-1, size-1)]:
                if random.random() < obstacle_prob:
                    grid[x][y].type = 'obstacle'
    
    return grid

grid = create_grid()  # Creates 10x10 grid 

In [217]:
#Visualization Grid
print(np.array([[cell.type[0] for cell in row] for row in grid]))

[['s' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'o']
 ['o' 'e' 'e' 'o' 'o' 'o' 'e' 'e' 'e' 'e']
 ['e' 'e' 'o' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['o' 'o' 'e' 'e' 'e' 'e' 'o' 'e' 'e' 'o']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'e' 'o']
 ['e' 'e' 'e' 'o' 'e' 'e' 'e' 'o' 'e' 'e']
 ['e' 'e' 'e' 'e' 'e' 'e' 'e' 'o' 'e' 'o']
 ['e' 'e' 'e' 'e' 'e' 'e' 'o' 'e' 'e' 'e']
 ['e' 'e' 'o' 'e' 'e' 'e' 'e' 'e' 'e' 'e']
 ['e' 'e' 'o' 'e' 'e' 'e' 'e' 'e' 'e' 'g']]


### Add Pathfinding Utilities

In [218]:
def is_valid(x, y, size=10):
    #Check if coordinates are within grid bounds
    return 0 <= x < size and 0 <= y < size


def get_neighbors(cell, grid):
    #Returns walkable adjacent cells (4-directional)
    # 4-directional movement (NSEW)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  
    neighbors = []
    
    for dx, dy in directions:
        nx, ny = cell.x + dx, cell.y + dy
        # Check bounds and obstacle status
        if is_valid(nx, ny) and grid[nx][ny].type != 'obstacle':
            neighbors.append(grid[nx][ny])
            
    return neighbors

### Use Manhattan distance

In [219]:
def manhattan_distance(cell1, cell2):
    return abs(cell1.x - cell2.x) + abs(cell1.y - cell2.y)

### Breadth-First Search (BFS)

In [220]:
def bfs(grid, start, goal):
    
    # Initialize data structures
    queue = deque([start])       # FIFO queue for BFS
    visited = set([start])       # Track visited nodes
    parents = {start: None}      # For path reconstruction
    explored = []                # All visited nodes in order
    
    while queue:
        current = queue.popleft()
        explored.append(current)
        
        # Early exit if goal found
        if current.type == 'goal':
            break
            
        # Explore neighbors
        for neighbor in get_neighbors(current, grid):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                parents[neighbor] = current  # Track parent-child relationship

    # Reconstruct path (goal to start)
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = parents.get(current)  # Backtrack using parent pointers
        
    path.reverse()  # Reverse to get start->goal path
    
    return path if path[0] == start else [], explored

### Run BFS

In [221]:
start = grid[0][0]
goal = grid[9][9]
bfs_path, bfs_explored = bfs(grid, start, goal)

bfs_path, bfs_explored = bfs(grid, start, goal)
print("BFS Path:", [(cell.x, cell.y) for cell in bfs_path])

BFS Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (7, 9), (8, 9), (9, 9)]


 ### Implement A* Algorithm 

In [225]:
def a_star(grid, start, goal):
    # A* algorithm with Manhattan distance heuristic
    
    # Priority queue setup (f_cost, counter, node)
    open_set = []
    counter = 0  # Tie-breaker when f_costs are equal
    heapq.heappush(open_set, (0, counter, start))
    
    # Cost and path tracking
    g_costs = {start: 0}  # Actual cost from start
    parents = {start: None}
    explored = []  # Closed set
    
    while open_set:
        _, _, current = heapq.heappop(open_set)
        explored.append(current)
        
        # Early termination
        if current.type == 'goal':
            break
            
        # Explore neighbors
        for neighbor in get_neighbors(current, grid):
            # Calculate tentative cost (1 per step for 4-directional grid)
            tentative_g_cost = g_costs[current] + 1  
            
            # Found better path?
            if neighbor not in g_costs or tentative_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = tentative_g_cost
                f_cost = tentative_g_cost + manhattan_distance(neighbor, goal)
                counter += 1
                heapq.heappush(open_set, (f_cost, counter, neighbor))
                parents[neighbor] = current

    # Reconstruct path
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = parents.get(current)
    path.reverse()
    
    return (path if path and path[0] == start else [], explored)
    
# Run A* 
astar_path, astar_explored = a_star(grid, start, goal)

### Run A* 

In [226]:
astar_path, astar_explored = a_star(grid, start, goal)
print("A* Path:", [(cell.x, cell.y) for cell in astar_path])

A* Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (7, 9), (8, 9), (9, 9)]


### Algorithm Performance Comparison

In [227]:
import time

# BFS multiple runs
bfs_total_time = 0
num_runs = 1000
for _ in range(num_runs):
    start_time = time.perf_counter()
    bfs_path, bfs_explored = bfs(grid, start, goal)
    bfs_total_time += time.perf_counter() - start_time
bfs_avg_time = bfs_total_time / num_runs

# A* multiple runs
astar_total_time = 0
for _ in range(num_runs):
    start_time = time.perf_counter()
    astar_path, astar_explored = a_star(grid, start, goal)
    astar_total_time += time.perf_counter() - start_time
astar_avg_time = astar_total_time / num_runs

# Results print 
print("BFS Path:", [(cell.x, cell.y) for cell in bfs_path])
print("BFS Steps:", len(bfs_path) - 1)
print("BFS Average Time (s):", bfs_avg_time)
print("BFS Explored Nodes:", len(bfs_explored))
print("\nA* Path:", [(cell.x, cell.y) for cell in astar_path])
print("A* Steps:", len(astar_path) - 1)
print("A* Average Time (s):", astar_avg_time)
print("A* Explored Nodes:", len(astar_explored))

BFS Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (7, 9), (8, 9), (9, 9)]
BFS Steps: 18
BFS Average Time (s): 0.00012648650037590413
BFS Explored Nodes: 73

A* Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (7, 9), (8, 9), (9, 9)]
A* Steps: 18
A* Average Time (s): 5.731279985047877e-05
A* Explored Nodes: 33


### Create results table

In [22]:
!pip install tabulate

In [229]:
from tabulate import tabulate

results = [
    ["Algorithm", "Path Length", "Avg.Time (ms)", "Nodes Explored", "Efficiency"],
    ["BFS", len(bfs_path)-1, f"{bfs_avg_time*1000:.2f}", len(bfs_explored), f"{len(bfs_path)/len(bfs_explored):.1%}"],
    ["A*", len(astar_path)-1, f"{astar_avg_time*1000:.2f}", len(astar_explored), f"{len(astar_path)/len(astar_explored):.1%}"]
]

print(tabulate(results, headers="firstrow", tablefmt="github"))
print("\nPath Coordinates:")
print(f"BFS: {[(c.x, c.y) for c in bfs_path]}")
print(f"A*:  {[(c.x, c.y) for c in astar_path]}")

| Algorithm   |   Path Length |   Avg.Time (ms) |   Nodes Explored | Efficiency   |
|-------------|---------------|-----------------|------------------|--------------|
| BFS         |            18 |            0.13 |               73 | 26.0%        |
| A*          |            18 |            0.06 |               33 | 57.6%        |

Path Coordinates:
BFS: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (7, 9), (8, 9), (9, 9)]
A*:  [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (7, 9), (8, 9), (9, 9)]


In [55]:
!pip install tk

In [242]:
import tkinter as tk
from tkinter import ttk, messagebox
import heapq
from collections import deque
import random
import time

class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.type = 'empty'
        self.parent = None
        self.g_cost = float('inf')
        
    def __repr__(self):
        return f"Cell({self.x}, {self.y}, {self.type})"

def create_grid(size=10, obstacle_prob=0.2):
    grid = [[Cell(x, y) for y in range(size)] for x in range(size)]
    grid[0][0].type = 'start'
    grid[size-1][size-1].type = 'goal'
    
    for x in range(size):
        for y in range(size):
            if (x, y) != (0, 0) and (x, y) != (size-1, size-1):
                if random.random() < obstacle_prob:
                    grid[x][y].type = 'obstacle'
    return grid

def is_valid(x, y, size=10):
    return 0 <= x < size and 0 <= y < size

def get_neighbors(cell, grid):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    neighbors = []
    for dx, dy in directions:
        nx, ny = cell.x + dx, cell.y + dy
        if is_valid(nx, ny, len(grid)) and grid[nx][ny].type != 'obstacle':
            neighbors.append(grid[nx][ny])
    return neighbors

def manhattan_distance(cell1, cell2):
    return abs(cell1.x - cell2.x) + abs(cell1.y - cell2.y)

class PathfindingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Pathfinding Visualizer")
        self.size = 10
        self.cell_size = 40
        self.grid = create_grid(self.size)
        self.dragging = False
        self.drag_type = None
        self.visualization_speed = 50  # ms between steps
        self.animation_running = False

        # Canvas
        self.canvas = tk.Canvas(root, width=self.size*self.cell_size, 
                               height=self.size*self.cell_size, bg='white')
        self.canvas.pack(pady=10)
        
        # Event bindings
        self.canvas.bind("<Button-1>", self.on_canvas_click)
        self.canvas.bind("<B1-Motion>", self.on_canvas_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_canvas_release)

        # Controls Frame
        self.controls = ttk.Frame(root)
        self.controls.pack(pady=10)

        # Algorithm selection
        ttk.Label(self.controls, text="Algorithm:").grid(row=0, column=0)
        self.algorithm = ttk.Combobox(self.controls, values=["BFS", "A*"], state="readonly")
        self.algorithm.current(0)
        self.algorithm.grid(row=0, column=1)

        # Visualization speed
        ttk.Label(self.controls, text="Speed:").grid(row=0, column=2)
        self.speed_scale = ttk.Scale(self.controls, from_=10, to=500, value=50)
        self.speed_scale.grid(row=0, column=3)
        self.speed_scale.bind("<ButtonRelease-1>", self.update_speed)

        # Obstacle probability
        ttk.Label(self.controls, text="Obstacle %:").grid(row=1, column=0)
        self.obstacle_prob = ttk.Scale(self.controls, from_=0, to=50, value=20)
        self.obstacle_prob.grid(row=1, column=1)

        # Buttons
        ttk.Button(self.controls, text="Run", command=self.run_algorithm).grid(row=1, column=2, padx=5)
        ttk.Button(self.controls, text="Pause", command=self.pause_animation).grid(row=1, column=3, padx=5)
        ttk.Button(self.controls, text="Reset", command=self.reset_grid).grid(row=1, column=4)
        ttk.Button(self.controls, text="Clear", command=self.clear_path).grid(row=1, column=5)

        # Status
        self.status = ttk.Label(root, text="Ready", relief="sunken")
        self.status.pack(fill="x", pady=5)

        self.draw_grid()

    def update_speed(self, event=None):
        self.visualization_speed = int(self.speed_scale.get())

    def on_canvas_click(self, event):
        if self.animation_running:
            return
            
        x, y = event.x // self.cell_size, event.y // self.cell_size
        if not (0 <= x < self.size and 0 <= y < self.size):
            return

        cell = self.grid[x][y]
        if cell.type == 'start':
            self.dragging = True
            self.drag_type = 'start'
        elif cell.type == 'goal':
            self.dragging = True
            self.drag_type = 'goal'
        elif cell.type == 'empty':
            cell.type = 'obstacle'
            self.draw_grid()
        elif cell.type == 'obstacle':
            cell.type = 'empty'
            self.draw_grid()

    def on_canvas_drag(self, event):
        if self.animation_running or not self.dragging:
            return

        x, y = event.x // self.cell_size, event.y // self.cell_size
        if not (0 <= x < self.size and 0 <= y < self.size):
            return

        cell = self.grid[x][y]
        if cell.type != 'obstacle':
            for row in self.grid:
                for c in row:
                    if c.type == self.drag_type:
                        c.type = 'empty'
            cell.type = self.drag_type
            self.draw_grid()

    def on_canvas_release(self, event):
        self.dragging = False
        self.drag_type = None

    def draw_grid(self, current=None, open_set=None, closed_set=None, path=None):
        self.canvas.delete("all")
        colors = {
            'empty': 'white',
            'start': 'green',
            'goal': 'red',
            'obstacle': 'black',
            'current': '#FFA500',  # Orange
            'open': '#ADD8E6',     # Light blue
            'closed': '#90EE90',   # Light green
            'path': 'blue'
        }
        
           
        for x in range(self.size):
            for y in range(self.size):
                x1 = x * self.cell_size
                y1 = y * self.cell_size
                x2 = x1 + self.cell_size
                y2 = y1 + self.cell_size
                
                cell = self.grid[x][y]
                fill = colors.get(cell.type, 'white')
                
                # Highlight different sets
                if path and cell in path:
                    fill = colors['path']
                elif current and cell == current:
                    fill = colors['current']
                elif open_set and cell in open_set:
                    fill = colors['open']
                elif closed_set and cell in closed_set:
                    fill = colors['closed']
                
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=fill, outline='gray')
                self.canvas.create_text(x1+self.cell_size//2, y1+self.cell_size//2, 
                                      text=f"{x},{y}", fill='black' if fill not in ['black', 'blue'] else 'white')
              
                
        


    def pause_animation(self):
        self.animation_running = False

    def run_algorithm(self):
        if self.animation_running:
            return
            
        start = None
        goal = None
        
        for row in self.grid:
            for cell in row:
                if cell.type == 'start':
                    start = cell
                elif cell.type == 'goal':
                    goal = cell
        
        if not start or not goal:
            messagebox.showerror("Error", "Please set both start and goal positions")
            return

        algo = self.algorithm.get()
        self.status.config(text=f"Running {algo}...")
        self.root.update()
        
        if algo == "BFS":
            self.animate_bfs(start, goal)
        else:
            self.animate_astar(start, goal)

    def animate_bfs(self, start, goal):
        self.animation_running = True
        queue = deque([start])
        visited = set([start])
        parents = {start: None}
        explored = []
        found = False

        while queue and self.animation_running:
            current = queue.popleft()
            explored.append(current)
            
            # Visualize current step
            self.draw_grid(current=current, closed_set=explored)
            self.status.config(text=f"BFS - Exploring: ({current.x},{current.y}) | Queue: {len(queue)}")
            self.root.update()
            time.sleep(self.visualization_speed/1000)
            
            if current == goal:
                found = True
                break
                
            for neighbor in get_neighbors(current, self.grid):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    parents[neighbor] = current
                    # Visualize newly discovered nodes
                    self.draw_grid(current=current, closed_set=explored, open_set=queue)
                    self.root.update()
                    time.sleep(self.visualization_speed/2000)

        path = []
        if found:
            current = goal
            while current is not None:
                path.append(current)
                current = parents.get(current)
            path.reverse()
            
            # Animate path reconstruction
            for i, cell in enumerate(path):
                self.draw_grid(closed_set=explored, path=path[:i+1])
                self.status.config(text=f"BFS - Path length: {i+1}")
                self.root.update()
                time.sleep(self.visualization_speed/1000)

        self.animation_running = False
        if path:
            self.status.config(text=f"BFS - Path found! Length: {len(path)} | Explored: {len(explored)}")
        else:
            self.status.config(text="BFS - No path found!")
            self.draw_grid(closed_set=explored)

    def animate_astar(self, start, goal):
        self.animation_running = True
        open_set = []
        counter = 0
        heapq.heappush(open_set, (0, counter, start))
        g_costs = {start: 0}
        parents = {start: None}
        explored = []
        found = False

        while open_set and self.animation_running:
            _, _, current = heapq.heappop(open_set)
            explored.append(current)
            
            # Visualize current step
            self.draw_grid(current=current, closed_set=explored)
            self.status.config(text=f"A* - Exploring: ({current.x},{current.y}) | g={g_costs[current]} | f={g_costs[current]+manhattan_distance(current, goal)}")
            self.root.update()
            time.sleep(self.visualization_speed/1000)
            
            if current == goal:
                found = True
                break
                
            for neighbor in get_neighbors(current, self.grid):
                tentative_g = g_costs[current] + 1
                
                if neighbor not in g_costs or tentative_g < g_costs[neighbor]:
                    g_costs[neighbor] = tentative_g
                    f_cost = tentative_g + manhattan_distance(neighbor, goal)
                    counter += 1
                    heapq.heappush(open_set, (f_cost, counter, neighbor))
                    parents[neighbor] = current
                    
                    # Visualize open set
                    open_cells = [cell for (_, _, cell) in open_set]
                    self.draw_grid(current=current, closed_set=explored, open_set=open_cells)
                    self.root.update()
                    time.sleep(self.visualization_speed/2000)

        path = []
        if found:
            current = goal
            while current is not None:
                path.append(current)
                current = parents.get(current)
            path.reverse()
            
            # Animate path reconstruction
            for i, cell in enumerate(path):
                self.draw_grid(closed_set=explored, path=path[:i+1])
                self.status.config(text=f"A* - Path length: {i+1} | Cost: {g_costs[cell]}")
                self.root.update()
                time.sleep(self.visualization_speed/1000)

        self.animation_running = False
        if path:
            self.status.config(text=f"A* - Path found! Length: {len(path)} | Explored: {len(explored)}")
        else:
            self.status.config(text="A* - No path found!")
            self.draw_grid(closed_set=explored)

    def reset_grid(self):
        if self.animation_running:
            self.pause_animation()
        prob = self.obstacle_prob.get() / 100
        self.grid = create_grid(self.size, prob)
        self.draw_grid()
        self.status.config(text=f"Grid reset with {prob:.0%} obstacles")

    def clear_path(self):
        if self.animation_running:
            self.pause_animation()
        self.draw_grid()
        self.status.config(text="Path cleared")

if __name__ == "__main__":
    root = tk.Tk()
    app = PathfindingApp(root)
    root.mainloop()